This notebook generates a list of all the unique incidents tracked in the SIT 209 report
unique_fires.csv contains:
1. Incident Number	
2. Fire Name	
3. First Day
4. Last Day	
5. Lat Fire	
6. Lon Fire

In [1]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import cartopy.crs as ccrs
import cartopy
import numpy as np
np.set_printoptions(threshold=33500)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union
from datetime import datetime, timedelta

In [18]:
#load in all the data
sit_all =pd.read_csv('../merged_sit.csv', dtype={'INCIDENT_JURISDICTION': 'str', 'UNIT_OR_OTHER_NARR': 'str'}) #all reports
#viirs_all = pd.read_csv('merged_viirs.csv', dtype={'satellite':'str', 'version': 'str'})
sit_history_2019 = pd.read_excel('SIT209_HISTORY_INCIDENTS_2019.xlsx') #incident summary
sit_history_2020 = pd.read_excel('SIT209_HISTORY_INCIDENTS_2020.xlsx')
sit_history_2021 = pd.read_excel('SIT209_HISTORY_INCIDENTS_2021.xlsx')

sit_history = pd.concat([sit_history_2019, sit_history_2020, sit_history_2021], axis = 0)

In [27]:
sit_history

,ID,INCIDENT_IDENTIFIER,INCIDENT_NUMBER,DONWCGU_PROT_UNIT_IDENTIFIER,INCIDENT_NAME,INCTYP_IDENTIFIER,CAUSE_IDENTIFIER,DISCOVERY_DATE,POO_LATITUDE,POO_LONGITUDE,...,EXT_SYSTEM_LAST_MODIFIED_DATE,FIRECODE,NWCG_PROT_UNIT_IDENTIFIER,LOCAL_TIMEZONE_IDENTIFIER,CY,COMPLEXITY_LEVEL_NARR,INCIDENT_COMMANDERS_NARR,COMPLEXITY_LEVEL_IDENTIFIER,UNIFIED_COMMAND_FLAG,INC_MGMT_ORG_IDENTIFIER
0,1,10665570,170.0,NaN,Renfro,9352235,9352170.0,2019-06-05 16:19:00,34.038100,-108.725600,...,2019-06-20 09:06:37,L92M,21052366.0,NaN,2019,NaN,Vowell,NaN,N,NaN
1,2,10665571,863.0,NaN,KIRKWOOD,9352235,9352170.0,2019-06-05 18:42:00,38.126940,-118.922900,...,2019-08-21 15:07:39,L92H,21054622.0,NaN,2019,NaN,MARK FOGG,NaN,N,NaN
2,3,10665572,168217.0,NaN,LAC-168217,9352235,9352172.0,2019-06-05 19:04:33,34.805400,-118.170800,...,2019-06-05 19:12:35,NaN,21054861.0,NaN,2019,NaN,NaN,NaN,N,NaN
3,4,10665573,143.0,NaN,Federal 2,9352235,9352170.0,2019-06-05 18:54:00,36.934780,-107.742600,...,2019-06-11 20:06:32,L92E,21052372.0,NaN,2019,NaN,Alfred Abreu,NaN,N,NaN
4,5,10665574,168231.0,NaN,LAC-168231,9352235,9352172.0,2019-06-05 19:19:01,34.704350,-118.083690,...,2019-06-05 19:24:40,NaN,21054861.0,NaN,2019,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44612,44613,13025129,004253,NaN,LAGOON,9608232,9608167.0,11/29/2021 12:18:00 AM,34.517310,-118.589700,...,12/27/2021 11:54:05 AM,PDG0,31163170.0,NaN,2021,NaN,DIV 1,NaN,N,NaN
44613,44614,13025130,381628,NaN,LAC-381628,9608232,9608169.0,11/29/2021 1:35:32 AM,34.587280,-118.099470,...,11/29/2021 10:09:00 AM,NaN,31163073.0,NaN,2021,NaN,NaN,NaN,N,NaN
44614,44615,13025131,165869,NaN,RIVERBOTTOM,9608232,9608169.0,11/28/2021 10:42:12 PM,33.985665,-117.400546,...,11/29/2021 10:09:00 AM,NaN,31163272.0,NaN,2021,NaN,NaN,NaN,N,NaN
44615,44616,13025132,004254,NaN,HARWOOD,9608232,9608169.0,11/29/2021 1:38:00 AM,34.282680,-117.612400,...,12/27/2021 11:54:05 AM,PDG1,31163170.0,NaN,2021,NaN,BC 21,NaN,N,NaN


In [19]:
sit_all.columns.values

array(['ID', 'INC209R_IDENTIFIER', 'INC_IDENTIFIER',
       'DONWCGU_PROT_UNIT_IDENTIFIER', 'REPORT_FROM_DATE',
       'REPORT_TO_DATE', 'STATUS', 'INCIDENT_DESCRIPTION',
       'INCIDENT_COMMANDERS_NARR', 'CURR_INCIDENT_AREA',
       'CURR_INC_AREA_UOM_IDENTIFIER', 'PCT_CONTAINED_COMPLETED',
       'PCT_CONT_COMPL_UOM_IDENTIFIER', 'COMPLEXITY_LEVEL_IDENTIFIER',
       'CAUSE_IDENTIFIER', 'POO_LATITUDE', 'POO_LONGITUDE',
       'POO_STATE_CODE', 'POO_CITY', 'POO_SHORT_LOCATION_DESC',
       'POO_US_NGR_ZONE', 'POO_US_NGR_XCOORD', 'POO_US_NGR_YCOORD',
       'POO_LD_TWP', 'POO_LD_RGE', 'POO_LD_SEC', 'POO_LD_QTR_SEC',
       'POO_LD_QTR_QTR_SEC', 'POO_LD_QTR_QTR_QTR_SEC',
       'POO_LD_QTR_QTR_QTR_QTR_SEC', 'POO_UTM_ZONE', 'POO_UTM_EASTING',
       'POO_UTM_NORTHING', 'INCIDENT_JURISDICTION',
       'ELEC_GEOSP_DATA_INCL', 'SIGNIF_EVENTS_SUMMARY',
       'FUEL_MODEL_IDENTIFIER', 'HAZARDS_MATLS_INVOLVMENT_NARR',
       'DAMAGE_ASSESSMENT_INFO', 'LIFE_SAFETY_HEALTH_STATUS_NARR',
       'W

In [16]:
print(sit_history_2019.columns.values)
print(sit_history_2020.columns.values)
print(sit_history_2021.columns.values)

['ID' 'INCIDENT_IDENTIFIER' 'INCIDENT_NUMBER'
 'DONWCGU_PROT_UNIT_IDENTIFIER' 'INCIDENT_NAME' 'INCTYP_IDENTIFIER'
 'CAUSE_IDENTIFIER' 'DISCOVERY_DATE' 'POO_LATITUDE' 'POO_LONGITUDE'
 'POO_STATE_CODE' 'POO_CITY' 'POO_SHORT_LOCATION_DESC' 'POO_US_NGR_ZONE'
 'POO_US_NGR_XCOORD' 'POO_US_NGR_YCOORD' 'POO_LD_TWP' 'POO_LD_RGE'
 'POO_LD_SEC' 'POO_LD_QTR_SEC' 'POO_LD_QTR_QTR_SEC'
 'POO_LD_QTR_QTR_QTR_SEC' 'POO_LD_QTR_QTR_QTR_QTR_SEC' 'POO_UTM_ZONE'
 'POO_UTM_EASTING' 'POO_UTM_NORTHING' 'INCIDENT_AREA'
 'INC_AREA_UOM_IDENTIFIER' 'EST_IM_COST_TO_DATE' 'PCT_CONTAINED_COMPLETED'
 'PCT_CONT_COMPL_UOM_IDENTIFIER' 'POO_COUNTY_CODE' 'CREATED_BY'
 'CREATED_DATE' 'LAST_MODIFIED_BY' 'LAST_MODIFIED_DATE'
 'PLANNED_START_DATE' 'PLANNED_DURATION' 'PLANNED_DUR_UOM_IDENTIFIER'
 'PLANNED_INCIDENT_SIZE' 'PLANNED_INC_SZ_UOM_IDENTIFIER'
 'POO_DONWCGU_OWN_IDENTIFIER' 'POO_LD_PM_IDENTIFIER' 'SINGLE_COMPLEX_FLAG'
 'IRWIN_IDENTIFIER' 'EXT_SYSTEM_LAST_MODIFIED_BY'
 'EXT_SYSTEM_LAST_MODIFIED_DATE' 'FIRECODE' 'NWCG_PROT_

In [9]:
sit_history

,ID,INCIDENT_IDENTIFIER,INCIDENT_NUMBER,DONWCGU_PROT_UNIT_IDENTIFIER,INCIDENT_NAME,INCTYP_IDENTIFIER,CAUSE_IDENTIFIER,DISCOVERY_DATE,POO_LATITUDE,POO_LONGITUDE,...,EXT_SYSTEM_LAST_MODIFIED_DATE,FIRECODE,NWCG_PROT_UNIT_IDENTIFIER,LOCAL_TIMEZONE_IDENTIFIER,CY,COMPLEXITY_LEVEL_NARR,INCIDENT_COMMANDERS_NARR,COMPLEXITY_LEVEL_IDENTIFIER,UNIFIED_COMMAND_FLAG,INC_MGMT_ORG_IDENTIFIER
0,1,10665570,170.0,NaN,Renfro,9352235,9352170.0,2019-06-05 16:19:00,34.038100,-108.725600,...,2019-06-20 09:06:37,L92M,21052366.0,NaN,2019,NaN,Vowell,NaN,N,NaN
1,2,10665571,863.0,NaN,KIRKWOOD,9352235,9352170.0,2019-06-05 18:42:00,38.126940,-118.922900,...,2019-08-21 15:07:39,L92H,21054622.0,NaN,2019,NaN,MARK FOGG,NaN,N,NaN
2,3,10665572,168217.0,NaN,LAC-168217,9352235,9352172.0,2019-06-05 19:04:33,34.805400,-118.170800,...,2019-06-05 19:12:35,NaN,21054861.0,NaN,2019,NaN,NaN,NaN,N,NaN
3,4,10665573,143.0,NaN,Federal 2,9352235,9352170.0,2019-06-05 18:54:00,36.934780,-107.742600,...,2019-06-11 20:06:32,L92E,21052372.0,NaN,2019,NaN,Alfred Abreu,NaN,N,NaN
4,5,10665574,168231.0,NaN,LAC-168231,9352235,9352172.0,2019-06-05 19:19:01,34.704350,-118.083690,...,2019-06-05 19:24:40,NaN,21054861.0,NaN,2019,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44612,44613,13025129,004253,NaN,LAGOON,9608232,9608167.0,11/29/2021 12:18:00 AM,34.517310,-118.589700,...,12/27/2021 11:54:05 AM,PDG0,31163170.0,NaN,2021,NaN,DIV 1,NaN,N,NaN
44613,44614,13025130,381628,NaN,LAC-381628,9608232,9608169.0,11/29/2021 1:35:32 AM,34.587280,-118.099470,...,11/29/2021 10:09:00 AM,NaN,31163073.0,NaN,2021,NaN,NaN,NaN,N,NaN
44614,44615,13025131,165869,NaN,RIVERBOTTOM,9608232,9608169.0,11/28/2021 10:42:12 PM,33.985665,-117.400546,...,11/29/2021 10:09:00 AM,NaN,31163272.0,NaN,2021,NaN,NaN,NaN,N,NaN
44615,44616,13025132,004254,NaN,HARWOOD,9608232,9608169.0,11/29/2021 1:38:00 AM,34.282680,-117.612400,...,12/27/2021 11:54:05 AM,PDG1,31163170.0,NaN,2021,NaN,BC 21,NaN,N,NaN


In [20]:
#generate the wildfire and complex codes
wildfire_codes = []
complex_codes = []
name_lookup ='SIT209_HISTORY_SIT209_LOOKUP_CODES_*.xlsx'
for year in range(2019, 2022):
    df_lookup = pd.read_excel(name_lookup.replace('*', str(year)))
    print(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values)
    wildfire_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Wildfire']['LUCODES_IDENTIFIER'].values[0])
    complex_codes.append(df_lookup.loc[df_lookup['CODE_NAME'] == 'Complex']['LUCODES_IDENTIFIER'].values[0])
print(wildfire_codes, complex_codes)

#select the wildfires and complexes only
sit_fire = sit_history[(sit_history['INCTYP_IDENTIFIER']==wildfire_codes[0])|(sit_history['INCTYP_IDENTIFIER']==wildfire_codes[1])|(sit_history['INCTYP_IDENTIFIER']==complex_codes[0])|(sit_history['INCTYP_IDENTIFIER']==complex_codes[1])]
print('We have ' + str(len(sit_fire['INCTYP_IDENTIFIER'])) + ' wildfire and complex reports')
#sanity check to see if we filtered out the right thing
aux = sit_history[(sit_history['INCTYP_IDENTIFIER']!=wildfire_codes[0])&(sit_history['INCTYP_IDENTIFIER']!=wildfire_codes[1])&(sit_history['INCTYP_IDENTIFIER']!=complex_codes[0])&(sit_history['INCTYP_IDENTIFIER']!=complex_codes[1])]
print(str(len(aux['INCTYP_IDENTIFIER']))+ ' Incidents were removed')

# find the unique incidents
inc_nums = sit_fire['INCIDENT_IDENTIFIER'].values # a numpy array of all the incidents
inc_names = sit_fire['INCIDENT_NAME'].values
inc_start_dates = sit_fire['DISCOVERY_DATE'].values

print(inc_nums)
print(inc_names)
print(inc_start_dates)
#compare number of unique incident numbers, names, and start dates
print('We have ' + str(len(np.unique(inc_nums))) + ' unique incident numbers')
#print('We have ' + str(len(np.unique(inc_names)))+ ' unique incident names, and ')  
#print('We have ' + str(len(np.unique(inc_start_dates)))+ ' unique start dates')  

[9352235]
[9429964]
[9608232]
[9352235, 9429964, 9608232] [9352456, 9430185, 9608453]
We have 63579 wildfire and complex reports
44986 Incidents were removed
[10665570 10665571 10665572 ... 11977512 11977513 11977514]
['Renfro' 'KIRKWOOD' 'LAC-168217' ... 'Rusk 14:51' 'LAC-345743'
 'John Gulch']
[Timestamp('2019-06-05 16:19:00') Timestamp('2019-06-05 18:42:00')
 Timestamp('2019-06-05 19:04:33') ... '11/15/2020 2:51:34 PM'
 '11/15/2020 6:41:07 PM' '12/18/2020 2:16:00 PM']
We have 63579 unique incident numbers


In [25]:
# generate a dataframe of incidents that contains:
# the name, number, earliest discovery date as a day, end date of final report as a day

unique_incs = np.unique(inc_nums)
n_incs = len(unique_incs) #number of unique incidents
print(n_incs)
incidents = pd.DataFrame({'Incident Number': np.nan*np.zeros(n_incs), 
                          'irwinID':np.nan*np.zeros(n_incs),
                          'Fire Name': np.nan*np.zeros(n_incs), 
                         'First Day': np.nan*np.zeros(n_incs),
                        'Last Day': np.nan*np.zeros(n_incs),
                        'Lat Fire': np.nan*np.zeros(n_incs), 
                     'Lon Fire': np.nan*np.zeros(n_incs),
                        'Final Area (ac)':np.nan*np.zeros(n_incs)}) # pre-allocate the dataframe

for ii in range(n_incs):
    #grab the name and incident number
    thisinc_num = unique_incs[ii] # Incident Number
    print(thisinc_num)
    df_fire = sit_fire[sit_fire['INCIDENT_IDENTIFIER']==thisinc_num]
    #df_fire = df_fire.sort_values('REPORT_FROM_DATE')
    thisinc_startdate = np.unique(df_fire['DISCOVERY_DATE'].values)
    thisinc_firstday = np.amin(thisinc_startdate)#[0:10] # First Day
    
    print(thisinc_startdate, thisinc_firstday)
    thisinc_enddate = df_fire['REPORT_TO_DATE'].values
    thisinc_lat = df_fire['POO_LATITUDE'].values[0]
    thisinc_lon = df_fire['POO_LONGITUDE'].values[0]
    
    #add the irwinid
    thisinc_irwinid = str(sit_history['IRWIN_IDENTIFIER'][sit_history['INCIDENT_IDENTIFIER']==thisinc_num].values)
    
    #GET THE AREA
    thisinc_area_array = sit_history['INCIDENT_AREA'][sit_history['INCIDENT_IDENTIFIER']==thisinc_num].values
    if len(thisinc_area_array)==0:
        thisinc_area=np.nan
    else:
        thisinc_area=thisinc_area_array[0]
    #print(thisinc_area)

    if len(thisinc_enddate)>0:
        thisinc_lastday = np.amax(thisinc_enddate)[0:10] #get the last available day
    else:
        thisinc_lastday = np.nan
    
    #thisinc_lastday = thisinc_enddate[0:10] # Last Day
    thisinc_name = np.unique(df_fire['INCIDENT_NAME'].values) #Fire Name
    
    #fill in the whole row at once (format below)
    incidents.iloc[ii]= [thisinc_num, thisinc_irwinid, thisinc_name[0], thisinc_firstday, thisinc_lastday, thisinc_lat, thisinc_lon,thisinc_area]
    
print(incidents)

63579
10662046
[Timestamp('2018-07-16 11:00:00')] 2018-07-16 11:00:00
Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACT

Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_72, PROJECTED_ACTIVITY_GT72, CURRENT_THREAT_12, CURRENT_THREAT_2

Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_72, PROJECTED_ACTIVITY_GT72, CURRENT_THREAT_12, CURRENT_THREAT_2

Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_72, PROJECTED_ACTIVITY_GT72, CURRENT_THREAT_12, CURRENT_THREAT_2

10662136
[Timestamp('2019-04-16 19:50:00')] 2019-04-16 19:50:00
Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_

...

10663582
[Timestamp('2019-05-14 10:48:21')] 2019-05-14 10:48:21
Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_

Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_72, PROJECTED_ACTIVITY_GT72, CURRENT_THREAT_12, CURRENT_THREAT_2

10663629
[Timestamp('2019-05-14 17:07:00')] 2019-05-14 17:07:00
Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_

Empty DataFrame
Columns: [ID, INC209R_IDENTIFIER, INC_IDENTIFIER, DONWCGU_PROT_UNIT_IDENTIFIER, REPORT_FROM_DATE, REPORT_TO_DATE, STATUS, INCIDENT_DESCRIPTION, INCIDENT_COMMANDERS_NARR, CURR_INCIDENT_AREA, CURR_INC_AREA_UOM_IDENTIFIER, PCT_CONTAINED_COMPLETED, PCT_CONT_COMPL_UOM_IDENTIFIER, COMPLEXITY_LEVEL_IDENTIFIER, CAUSE_IDENTIFIER, POO_LATITUDE, POO_LONGITUDE, POO_STATE_CODE, POO_CITY, POO_SHORT_LOCATION_DESC, POO_US_NGR_ZONE, POO_US_NGR_XCOORD, POO_US_NGR_YCOORD, POO_LD_TWP, POO_LD_RGE, POO_LD_SEC, POO_LD_QTR_SEC, POO_LD_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_SEC, POO_LD_QTR_QTR_QTR_QTR_SEC, POO_UTM_ZONE, POO_UTM_EASTING, POO_UTM_NORTHING, INCIDENT_JURISDICTION, ELEC_GEOSP_DATA_INCL, SIGNIF_EVENTS_SUMMARY, FUEL_MODEL_IDENTIFIER, HAZARDS_MATLS_INVOLVMENT_NARR, DAMAGE_ASSESSMENT_INFO, LIFE_SAFETY_HEALTH_STATUS_NARR, WEATHER_CONCERNS_NARR, PROJECTED_ACTIVITY_12, PROJECTED_ACTIVITY_24, PROJECTED_ACTIVITY_48, PROJECTED_ACTIVITY_72, PROJECTED_ACTIVITY_GT72, CURRENT_THREAT_12, CURRENT_THREAT_2

KeyboardInterrupt: 

In [17]:
print(incidents[incidents['Fire Name']=='CZU AUG LIGHTNING'])

      Incident Number                                   irwinID  \
1129       11878096.0  ['947FBFB1-EBEA-452F-9AD5-E4FC38C05068']   

              Fire Name            First Day    Last Day   Lat Fire  \
1129  CZU AUG LIGHTNING  2020-08-16 08:41:00  2020-09-22  37.261667   

        Lon Fire  Final Area (ac)  
1129 -122.223333          86509.0  


In [18]:
#check for nan values of the fire location, and drop them!
te = incidents['Lat Fire'].values
drops = np.where(np.isnan(te))[0]
incidents = incidents.drop(labels=drops)

In [19]:
# filter out fires whose starting location is outside of our domain
incidents = incidents[(incidents['Lat Fire']>=31)&(incidents['Lat Fire']<=49)&(incidents['Lon Fire']>=-125)&(incidents['Lon Fire']<=-101)]
print(incidents)

      Incident Number      Fire Name            First Day    Last Day  \
0          10662156.0            C34  2019-02-13 15:00:00  2019-02-15   
1          10662179.0           2161  2019-02-13 16:40:00  2019-02-14   
2          10662684.0  PAINTED WAGON  2019-04-08 13:02:00  2019-04-10   
3          10663171.0  LONE MOUNTAIN  2019-05-26 23:56:00  2019-07-03   
4          10663525.0        BOULDER  2019-06-05 12:45:00  2019-06-06   
...               ...            ...                  ...         ...   
1350       11979081.0        LAURA 2  2020-11-17 13:30:00  2020-11-23   
1351       11979237.0          Trail  2020-10-24 21:32:00  2020-11-02   
1352       11980186.0      SANDERSON  2020-12-13 03:00:00  2020-12-14   
1353       11980587.0        CREEK 5  2020-12-24 01:22:00  2020-12-28   
1354       11980949.0        CREEK 5  2020-12-24 01:00:00  2020-12-24   

       Lat Fire    Lon Fire  Final Area (ac)  
0     36.995278 -102.524722           3812.1  
1     35.221389 -101.518611  

In [18]:
# save incidents as a csv file
#incidents.to_csv('unique_fires.csv', index=False)
#incidents.to_csv('unique_fires_with_area.csv', index=False)
incidents.to_csv('unique_fires_with_area_and_irwin.csv', index=False)